In [1]:
from sqlalchemy import text
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

Hello World (SQL text command)

In [2]:
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2023-08-06 13:38:52,171 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:38:52,172 INFO sqlalchemy.engine.Engine select 'hello world'
2023-08-06 13:38:52,174 INFO sqlalchemy.engine.Engine [generated in 0.00326s] ()
[('hello world',)]
2023-08-06 13:38:52,176 INFO sqlalchemy.engine.Engine ROLLBACK


In [3]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-08-06 13:38:52,188 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:38:52,189 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-08-06 13:38:52,191 INFO sqlalchemy.engine.Engine [generated in 0.00275s] ()
2023-08-06 13:38:52,193 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-08-06 13:38:52,195 INFO sqlalchemy.engine.Engine [generated in 0.00199s] [(1, 1), (2, 4)]
2023-08-06 13:38:52,196 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.connect() as conn:
    result = conn.execute(text("select x,y from some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-08-06 13:38:52,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:38:52,215 INFO sqlalchemy.engine.Engine select x,y from some_table
2023-08-06 13:38:52,216 INFO sqlalchemy.engine.Engine [generated in 0.00328s] ()
x: 1  y: 1
x: 2  y: 4
2023-08-06 13:38:52,218 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-08-06 13:38:52,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:38:52,234 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-08-06 13:38:52,235 INFO sqlalchemy.engine.Engine [cached since 0.04175s ago] [(6, 8), (9, 10)]
2023-08-06 13:38:52,236 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with engine.connect() as conn:
    result = conn.execute(text("select x,y from some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-08-06 13:39:09,150 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:39:09,151 INFO sqlalchemy.engine.Engine select x,y from some_table
2023-08-06 13:39:09,152 INFO sqlalchemy.engine.Engine [cached since 16.94s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-08-06 13:39:09,153 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y AND x > :x"), {"y": 4, "x":6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-08-06 13:41:20,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:41:20,330 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? AND x > ?
2023-08-06 13:41:20,331 INFO sqlalchemy.engine.Engine [cached since 29.34s ago] (4, 6)
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2023-08-06 13:41:20,334 INFO sqlalchemy.engine.Engine ROLLBACK


In [14]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2023-08-06 13:41:15,695 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:41:15,697 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-08-06 13:41:15,698 INFO sqlalchemy.engine.Engine [cached since 143.5s ago] [(11, 12), (13, 14)]
2023-08-06 13:41:15,701 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-08-06 13:43:55,052 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:43:55,055 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-08-06 13:43:55,057 INFO sqlalchemy.engine.Engine [cached since 46.32s ago] (6,)
x: 6  y: 8
x: 9  y: 11
x: 11  y: 12
x: 13  y: 15
2023-08-06 13:43:55,061 INFO sqlalchemy.engine.Engine ROLLBACK


In [17]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2023-08-06 13:43:47,766 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:43:47,768 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2023-08-06 13:43:47,769 INFO sqlalchemy.engine.Engine [generated in 0.00144s] [(11, 9), (15, 13)]
2023-08-06 13:43:47,771 INFO sqlalchemy.engine.Engine COMMIT


In [21]:
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import MetaData
metadata_obj = MetaData()
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2023-08-06 13:53:43,872 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-06 13:53:43,874 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2023-08-06 13:53:43,876 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-06 13:53:43,878 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-08-06 13:53:43,880 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2023-08-06 13:53:43,882 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2023-08-06 13:53:43,883 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-06 13:53:43,884 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2023-08-06 13:53:43,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-08-06 13:53:43,886 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [22]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)